In [10]:
import pandas as pd
import numpy as np
import sys

import warnings
import math
warnings.filterwarnings('ignore')

ACTION_TYPE_NONE = 0
ACTION_TYPE_BUY = 1
ACTION_TYPE_SELL = 2
ACTION_TYPE_HOLD = 3


df=pd.read_csv('i_1min_2013_2017.csv', sep=',',header=0)
prices = df.values[-225:, [0,1,2,3,4,5,6,7]]
print(prices[-10:])



[['2017/12/29' '14:51' 528.0 528.0 527.0 528.0 6692 1748776]
 ['2017/12/29' '14:52' 528.0 528.0 527.5 528.0 5032 1747540]
 ['2017/12/29' '14:53' 527.5 529.0 527.5 529.0 13256 1749290]
 ['2017/12/29' '14:54' 529.0 529.5 528.5 529.0 11992 1747568]
 ['2017/12/29' '14:55' 529.0 531.0 528.5 531.0 28436 1745718]
 ['2017/12/29' '14:56' 531.0 531.0 529.0 529.5 20080 1748302]
 ['2017/12/29' '14:57' 529.5 530.0 529.0 530.0 10138 1749830]
 ['2017/12/29' '14:58' 529.5 530.0 529.0 529.0 11250 1752602]
 ['2017/12/29' '14:59' 529.0 530.0 529.0 530.0 10748 1751228]
 ['2017/12/29' '15:00' 530.0 532.0 529.5 531.5 36668 1750082]]


In [11]:
print(df['Date'] + ' ' + df['Time'])

0            2014/1/2 9:01
1            2014/1/2 9:02
2            2014/1/2 9:03
3            2014/1/2 9:04
4            2014/1/2 9:05
5            2014/1/2 9:06
6            2014/1/2 9:07
7            2014/1/2 9:08
8            2014/1/2 9:09
9            2014/1/2 9:10
10           2014/1/2 9:11
11           2014/1/2 9:12
12           2014/1/2 9:13
13           2014/1/2 9:14
14           2014/1/2 9:15
15           2014/1/2 9:16
16           2014/1/2 9:17
17           2014/1/2 9:18
18           2014/1/2 9:19
19           2014/1/2 9:20
20           2014/1/2 9:21
21           2014/1/2 9:22
22           2014/1/2 9:23
23           2014/1/2 9:24
24           2014/1/2 9:25
25           2014/1/2 9:26
26           2014/1/2 9:27
27           2014/1/2 9:28
28           2014/1/2 9:29
29           2014/1/2 9:30
                ...       
337647    2017/12/29 14:31
337648    2017/12/29 14:32
337649    2017/12/29 14:33
337650    2017/12/29 14:34
337651    2017/12/29 14:35
337652    2017/12/29 14:36
3

In [12]:
import time
import datetime

df['timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%Y%m%d %H:%M")
print(df['timestamp'])

df['timedelta'] = (df['timestamp'] - df['timestamp'].shift()).fillna(10000).dt.total_seconds()
print(df['timedelta'][128:])


split_index_list = (df.index[df['timedelta'] > 3600]).tolist()
print(split_index_list)

split_df = [df.iloc[split_index_list[n]:split_index_list[n+1]] for n in range(len(split_index_list)-1)]

0        2014-01-02 09:01:00
1        2014-01-02 09:02:00
2        2014-01-02 09:03:00
3        2014-01-02 09:04:00
4        2014-01-02 09:05:00
5        2014-01-02 09:06:00
6        2014-01-02 09:07:00
7        2014-01-02 09:08:00
8        2014-01-02 09:09:00
9        2014-01-02 09:10:00
10       2014-01-02 09:11:00
11       2014-01-02 09:12:00
12       2014-01-02 09:13:00
13       2014-01-02 09:14:00
14       2014-01-02 09:15:00
15       2014-01-02 09:16:00
16       2014-01-02 09:17:00
17       2014-01-02 09:18:00
18       2014-01-02 09:19:00
19       2014-01-02 09:20:00
20       2014-01-02 09:21:00
21       2014-01-02 09:22:00
22       2014-01-02 09:23:00
23       2014-01-02 09:24:00
24       2014-01-02 09:25:00
25       2014-01-02 09:26:00
26       2014-01-02 09:27:00
27       2014-01-02 09:28:00
28       2014-01-02 09:29:00
29       2014-01-02 09:30:00
                 ...        
337647   2017-12-29 14:31:00
337648   2017-12-29 14:32:00
337649   2017-12-29 14:33:00
337650   2017-

In [13]:
# make data for last 200 days.
# data is in the shape [200, 375, 6]
# value is in the shape (200, 375, 2)


length_list = [len(item) for item in split_df]

steps     = max(length_list) - 1

print("maximum steps:{}".format(steps))

n_inputs  = 6
n_outputs = 2

ACTION_BUY = 0    # do more
ACTION_SELL = 1   # do short
ACTION_NONE = 2



def get_best_action(Q):
    return np.argmax(np.hstack((Q,[0])))

def get_best_action_value(Q):
    action_index = get_best_action(Q)
    return action_index, Q[action_index]

for gamma in (1,0.9,0.8,0.7,0.6):
    data = np.zeros((len(split_df), steps, 6))
    data_out = np.zeros((len(split_df), steps, 2))
    data_out_actions = np.zeros((len(split_df), steps))

    # preparing data
    for i in range(len(split_df)): # hist_days + test_days
        # the difference between open and last close.
        print("handling days: {}, len:{}".format(i, len(split_df[i])))
        length = len(split_df[i]) - 1

        data[i,:length, 0] = np.divide(np.subtract(split_df[i].values[1:, 3], split_df[i].values[:-1, 5]), split_df[i].values[:-1, 5])
        data[i,:length, 1] = np.divide(np.subtract(split_df[i].values[1:, 4], split_df[i].values[:-1, 5]), split_df[i].values[:-1, 5])
        data[i,:length, 2] = np.divide(np.subtract(split_df[i].values[1:, 4], split_df[i].values[:-1, 5]), split_df[i].values[:-1, 5])
        data[i,:length, 3] = np.divide(np.diff(split_df[i].values[:, 5]), split_df[i].values[:-1, 5])
        data[i,:length, 4] = split_df[i].values[1:,6]
        data[i,:length, 5] = split_df[i].values[1:,7]

        # calculate the value for the day using close_diff.
        close_diff = data[i,:,3]
        Q_values = np.zeros((steps,2))
        R = np.zeros((2))
        actions = np.zeros((steps))
        last_profit = 1
        for iter in range(30):
            profit = 1
            last_action = ACTION_NONE
            for t in range(steps - 1):
                R[ACTION_BUY] = close_diff[t + 1]
                R[ACTION_SELL] = -close_diff[t + 1]
                best_next_action, best_next_value = get_best_action_value(Q_values[t+1])
                if best_next_action == ACTION_BUY:
                    Q_values[t][ACTION_BUY]  = R[ACTION_BUY] + gamma * best_next_value
                    Q_values[t][ACTION_SELL] = R[ACTION_SELL]
                elif best_next_action == ACTION_SELL:
                    Q_values[t][ACTION_SELL] = R[ACTION_SELL] + gamma * best_next_value
                    Q_values[t][ACTION_BUY] = R[ACTION_BUY]
                else: # ACTION_NONE
                    Q_values[t][ACTION_BUY] = R[ACTION_BUY]
                    Q_values[t][ACTION_SELL] = R[ACTION_SELL]

                action, action_value = get_best_action_value(Q_values[t])

                profit = profit * (1 + R[action])
                last_action = action
                actions[t] = action
            #print("iter={} profit={}".format(iter, profit))
            #print(np.hstack((Q_values, actions.reshape(Q_values.shape[0], 1), close_diff.reshape(steps,1))))
            last_profit = profit
        print("sequence:{} finished, profit={}".format(i, profit))
        data_out[i] = Q_values
        data_out_actions[i] = actions
    # do standardlize, only for training data!

    # save data into file.
    data_all = np.concatenate((data, data_out), axis=2)
    filename = 'i_raw_data.bin_gamma_' + str(gamma)
    np.save(filename, data_all)



maximum steps:329
handling days: 0, len:129
sequence:0 finished, profit=1.027845339541625
handling days: 1, len:90
sequence:1 finished, profit=1.0222857858543368
handling days: 2, len:135
sequence:2 finished, profit=1.0224966713148496
handling days: 3, len:90
sequence:3 finished, profit=1.033919196770764
handling days: 4, len:135
sequence:4 finished, profit=1.0375152343488427
handling days: 5, len:90
sequence:5 finished, profit=1.0284098559696568
handling days: 6, len:134
sequence:6 finished, profit=1.0340646163451237
handling days: 7, len:89
sequence:7 finished, profit=1.0157249778923263
handling days: 8, len:134
sequence:8 finished, profit=1.0376670862459623
handling days: 9, len:88
sequence:9 finished, profit=1.0135260273445788
handling days: 10, len:131
sequence:10 finished, profit=1.034108523398996
handling days: 11, len:89
sequence:11 finished, profit=1.0112815061868166
handling days: 12, len:134
sequence:12 finished, profit=1.0371350560852584
handling days: 13, len:90
sequence:1

sequence:110 finished, profit=1.051149496721744
handling days: 111, len:90
sequence:111 finished, profit=1.0251426452027952
handling days: 112, len:135
sequence:112 finished, profit=1.0591850279830115
handling days: 113, len:90
sequence:113 finished, profit=1.0421347411808441
handling days: 114, len:135
sequence:114 finished, profit=1.0520493567932057
handling days: 115, len:90
sequence:115 finished, profit=1.0436887195467635
handling days: 116, len:135
sequence:116 finished, profit=1.0516589887059475
handling days: 117, len:90
sequence:117 finished, profit=1.0266404545741097
handling days: 118, len:135
sequence:118 finished, profit=1.0465226148868279
handling days: 119, len:90
sequence:119 finished, profit=1.0166286656096375
handling days: 120, len:135
sequence:120 finished, profit=1.0574479569519155
handling days: 121, len:90
sequence:121 finished, profit=1.0305908304650893
handling days: 122, len:135
sequence:122 finished, profit=1.057746366167974
handling days: 123, len:90
sequence

sequence:221 finished, profit=1.0460232234408509
handling days: 222, len:135
sequence:222 finished, profit=1.0476363353118312
handling days: 223, len:90
sequence:223 finished, profit=1.0475941829401234
handling days: 224, len:135
sequence:224 finished, profit=1.030097552023547
handling days: 225, len:90
sequence:225 finished, profit=1.0360909861185943
handling days: 226, len:135
sequence:226 finished, profit=1.0781040122483396
handling days: 227, len:90
sequence:227 finished, profit=1.0517902994975237
handling days: 228, len:135
sequence:228 finished, profit=1.0554899944463454
handling days: 229, len:90
sequence:229 finished, profit=1.0251378984787465
handling days: 230, len:135
sequence:230 finished, profit=1.0747903453894048
handling days: 231, len:90
sequence:231 finished, profit=1.0307851306109883
handling days: 232, len:135
sequence:232 finished, profit=1.0491792513269214
handling days: 233, len:90
sequence:233 finished, profit=1.0197495273201347
handling days: 234, len:135
sequen

sequence:329 finished, profit=1.0208765222837524
handling days: 330, len:135
sequence:330 finished, profit=1.0788744044073726
handling days: 331, len:90
sequence:331 finished, profit=1.0333422018974576
handling days: 332, len:135
sequence:332 finished, profit=1.0541558998794436
handling days: 333, len:90
sequence:333 finished, profit=1.0440842535543544
handling days: 334, len:135
sequence:334 finished, profit=1.0625962156027176
handling days: 335, len:90
sequence:335 finished, profit=1.0433363577466934
handling days: 336, len:135
sequence:336 finished, profit=1.0882786393152195
handling days: 337, len:90
sequence:337 finished, profit=1.0649829484400613
handling days: 338, len:135
sequence:338 finished, profit=1.0647454808068413
handling days: 339, len:90
sequence:339 finished, profit=1.0275077676712607
handling days: 340, len:135
sequence:340 finished, profit=1.1076606721423483
handling days: 341, len:90
sequence:341 finished, profit=1.04002886151762
handling days: 342, len:135
sequenc

sequence:437 finished, profit=1.0372718465147375
handling days: 438, len:135
sequence:438 finished, profit=1.0993395297484994
handling days: 439, len:90
sequence:439 finished, profit=1.028439876151805
handling days: 440, len:135
sequence:440 finished, profit=1.111727222414784
handling days: 441, len:90
sequence:441 finished, profit=1.0341228014194768
handling days: 442, len:135
sequence:442 finished, profit=1.0703395060850196
handling days: 443, len:90
sequence:443 finished, profit=1.0700508443122618
handling days: 444, len:135
sequence:444 finished, profit=1.1006256506774619
handling days: 445, len:90
sequence:445 finished, profit=1.0512439287686122
handling days: 446, len:135
sequence:446 finished, profit=1.0795630661178213
handling days: 447, len:90
sequence:447 finished, profit=1.0537209398774987
handling days: 448, len:135
sequence:448 finished, profit=1.065035420417318
handling days: 449, len:90
sequence:449 finished, profit=1.0338237764312261
handling days: 450, len:135
sequence

...

sequence:1943 finished, profit=1.1168101993347734
handling days: 1944, len:150
sequence:1944 finished, profit=1.1652379666899386
handling days: 1945, len:135
sequence:1945 finished, profit=1.1569207418430643
handling days: 1946, len:90
sequence:1946 finished, profit=1.096412322643187
handling days: 1947, len:150
sequence:1947 finished, profit=1.1596677624184712
handling days: 1948, len:135
sequence:1948 finished, profit=1.1216661275644981
handling days: 1949, len:90
sequence:1949 finished, profit=1.1010999190663855
handling days: 1950, len:150
sequence:1950 finished, profit=1.1522894459503088
handling days: 1951, len:135
sequence:1951 finished, profit=1.1402188146818177
handling days: 1952, len:90
sequence:1952 finished, profit=1.0976021145881556
handling days: 1953, len:150
sequence:1953 finished, profit=1.157460413071775
handling days: 1954, len:135
sequence:1954 finished, profit=1.140417214311586
handling days: 1955, len:90
sequence:1955 finished, profit=1.1199058636018022
handling 

sequence:2048 finished, profit=1.1632501000326714
handling days: 2049, len:135
sequence:2049 finished, profit=1.1488469087353494
handling days: 2050, len:90
sequence:2050 finished, profit=1.0843842928167182
handling days: 2051, len:150
sequence:2051 finished, profit=1.1632404201526023
handling days: 2052, len:135
sequence:2052 finished, profit=1.1405365116460857
handling days: 2053, len:90
sequence:2053 finished, profit=1.083816893365908
handling days: 2054, len:150
sequence:2054 finished, profit=1.1741257691088778
handling days: 2055, len:135
sequence:2055 finished, profit=1.1305844951122284
handling days: 2056, len:90
sequence:2056 finished, profit=1.1009234230352287
handling days: 2057, len:150
sequence:2057 finished, profit=1.1769665310671127
handling days: 2058, len:135
sequence:2058 finished, profit=1.1682220030559276
handling days: 2059, len:90
sequence:2059 finished, profit=1.118797715808602
handling days: 2060, len:150
sequence:2060 finished, profit=1.1410256744162754
handling

sequence:2257 finished, profit=1.1782368356515804
handling days: 2258, len:135
sequence:2258 finished, profit=1.171657809645331
handling days: 2259, len:90
sequence:2259 finished, profit=1.1061039341162051
handling days: 2260, len:150
sequence:2260 finished, profit=1.201590271247549
handling days: 2261, len:135
sequence:2261 finished, profit=1.1440032906254887
handling days: 2262, len:90
sequence:2262 finished, profit=1.08543543436896
handling days: 2263, len:150
sequence:2263 finished, profit=1.148316009688432
handling days: 2264, len:135
sequence:2264 finished, profit=1.135273253055373
handling days: 2265, len:90
sequence:2265 finished, profit=1.0944197055478238
handling days: 2266, len:150
sequence:2266 finished, profit=1.1206110731084276
handling days: 2267, len:135
sequence:2267 finished, profit=1.1119639422267527
handling days: 2268, len:90
sequence:2268 finished, profit=1.0908196579372462
handling days: 2269, len:150
sequence:2269 finished, profit=1.1677194258896362
handling day

sequence:2362 finished, profit=1.1727041110479572
handling days: 2363, len:135
sequence:2363 finished, profit=1.1475156911581479
handling days: 2364, len:90
sequence:2364 finished, profit=1.129823932386307
handling days: 2365, len:150
sequence:2365 finished, profit=1.1854693127054603
handling days: 2366, len:135
sequence:2366 finished, profit=1.1750879279244233
handling days: 2367, len:90
sequence:2367 finished, profit=1.0828298521809991
handling days: 2368, len:150
sequence:2368 finished, profit=1.1853538600970086
handling days: 2369, len:135
sequence:2369 finished, profit=1.277789356486836
handling days: 2370, len:90
sequence:2370 finished, profit=1.1282883764804341
handling days: 2371, len:150
sequence:2371 finished, profit=1.1864946709551454
handling days: 2372, len:135
sequence:2372 finished, profit=1.1675729169854465
handling days: 2373, len:90
sequence:2373 finished, profit=1.1159430587690105
handling days: 2374, len:150
sequence:2374 finished, profit=1.2035060456892852
handling

sequence:2572 finished, profit=1.1536193272518762
handling days: 2573, len:90
sequence:2573 finished, profit=1.0807835796666994
handling days: 2574, len:150
sequence:2574 finished, profit=1.177386698903141
handling days: 2575, len:135
sequence:2575 finished, profit=1.1340959896343745
handling days: 2576, len:90
sequence:2576 finished, profit=1.0863401914726873
handling days: 2577, len:150
sequence:2577 finished, profit=1.141084760238916
handling days: 2578, len:135
sequence:2578 finished, profit=1.1413318114115523
handling days: 2579, len:90
sequence:2579 finished, profit=1.1070718944993425
handling days: 2580, len:150
sequence:2580 finished, profit=1.1239438555454628
handling days: 2581, len:135
sequence:2581 finished, profit=1.1036567500774659
handling days: 2582, len:90
sequence:2582 finished, profit=1.0735621145926428
handling days: 2583, len:150
sequence:2583 finished, profit=1.1065123866056819
handling days: 2584, len:135
sequence:2584 finished, profit=1.1160453202397083
handling

In [268]:
import plotly.plotly as py
import plotly.graph_objs as go

actions = data_out_actions[0]
x = df_date[0].values[:,1]
def get_price_if_true(t, show):
    if show:
        return df_date[0].values[t+1, 5]
    else:
        return None
    
data0_y = [get_price_if_true(i, actions[i] == ACTION_BUY) for i in range(steps)]
data1_y = [get_price_if_true(i, actions[i] == ACTION_SELL) for i in range(steps)]
data2_y = [get_price_if_true(i, actions[i] == ACTION_NONE) for i in range(steps)]

# Create a trace
trace0 = go.Scatter(
    x = x,
    y = data0_y,
    connectgaps=False,
    name='buy'
)

trace1 = go.Scatter(
    x = x,
    y = data1_y,
    connectgaps=False,
    name='sell'
)

trace2 = go.Scatter(
    x = x,
    y = data2_y,
    connectgaps=False,
    name='do nothing'
)

data_graph = [trace0, trace1,trace2]
layout = go.Layout(
    title='',
    xaxis=dict(
        title='Time(s)',
    ),
    yaxis=dict(
        title='Price',
    )
)
fig = go.Figure(data=data_graph, layout=layout)
py.iplot(data_graph, filename='basic-line')

IndexError: index 225 is out of bounds for axis 0 with size 225

In [ ]:
x2, y2, z2 = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 200).transpose()
trace2 = go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers',
    marker=dict(
        color='rgb(127, 127, 127)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.9
    )
)